In [1]:
%pylab inline
import time
from collections import defaultdict
import json
import pandas as pd
import os
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout



import numpy as np
import requests
import json 
import datetime


from textblob import TextBlob

Populating the interactive namespace from numpy and matplotlib


In [185]:
with open("seed_creation_date.json","r") as f:
    sci_aw_dict=json.load(f,encoding="utf-8")
print len(sci_dict)

262


In [7]:
with open("baseline_creation_date.json","r") as f:
    sci_nonaw_dict=json.load(f,encoding="utf-8")
print len(sci_nonaw_dict)

276


#https://stackoverflow.com/questions/34411896/how-to-get-full-wikipedia-revision-history-list-from-some-article



http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&&titles=Akiko_Iwasaki
    
    
https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Bla_Bla_Bla&prop=extracts&exintro&explaintext
    
    exintro - parse first paragraph

# Get all revisions of Wikipedia articles

## Non awarded scientists

In [159]:
#pip install pywikibot
#create file user-config.py
#
## -*- coding: utf-8 -*-
#family = 'wikipedia'
#mylang = 'en'
#usernames['wikipedia']['en'] = u'RevisionsBot'
#
#
#
#https://doc.wikimedia.org/pywikibot/api_ref/pywikibot.html#module-pywikibot.page

import pywikibot
site = pywikibot.Site("en", "wikipedia")

In [160]:
list_rev=[]
erorr_list=[]
j=1
for sci_name in sci_nonaw_dict.keys():
    name=sci_name.split("/")[-1]
    print j,name
    try:
        page = pywikibot.Page(site, name)
        revs = page.revisions(content=True)
        for i in revs:
            temp_dict=i.__dict__
            temp_dict["article"]=name
            list_rev.append(temp_dict)
    except:
        print "Error when access:", name
        erorr_list.append(name)
        
    j+=1

1 G%C3%A1bor_A._Somorjai
2 Elias_James_Corey
3 Albert_Meyers
4 Dan_Boneh
5 Shlomo_Shamai
6 Xavier_Gabaix
7 Marianne_Bertrand
8 Malcolm_Green_(chemist)
9 Subir_Sachdev
10 Jing_Li_(chemist)
11 Steven_Kaplan_(economist)
12 John_Perdew
13 Arun_Majumdar
14 Giuseppe_Mingione
15 Menachem_Elimelech
16 Juan_Mart%C3%ADn_Maldacena
17 Theodore_H._Geballe
18 K%C3%A1ri_Stef%C3%A1nsson
19 Ian_Ford
20 Stephen_R._Bloom
21 Andrei_Shleifer
22 Ralph_Weissleder
23 Axel_D._Becke
24 John_Robertson_(physicist)
25 Matja%C5%BE_Perc
26 Ariel_Rubinstein
27 Stephen_Vargo
28 Michael_Boehnke
29 Oded_Schramm
30 Michael_I._Jordan
31 Sergio_Ferrara
32 Wayne_Fuller
33 Rudolf_Grimm
34 Joseph_Wang
35 Immanuel_Bloch
36 Andrew_Bernard
37 Cumrun_Vafa
38 George_Loewenstein
39 Dimitri_Nanopoulos
40 Joann_E._Manson
Error when access: Joann_E._Manson
41 Robert_J._Birgeneau
42 Pauline_van_den_Driessche
43 Murray_Brennan
44 Ralph_Nuzzo
45 Donal_Bradley
46 Paul_Ridker
47 Ulrike_Malmendier
48 Allen_Steere
49 Edward_Ott
50 Alexander_

In [168]:
print erorr_list

[u'Joann_E._Manson']


In [166]:
df=pd.DataFrame(list_rev)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24163 entries, 0 to 24162
Data columns (total 12 columns):
_content_model    24163 non-null object
_parent_id        24163 non-null int64
_sha1             24132 non-null object
anon              24163 non-null bool
article           24163 non-null object
comment           24163 non-null object
minor             24163 non-null bool
revid             24163 non-null int64
rollbacktoken     0 non-null object
text              24132 non-null object
timestamp         24163 non-null datetime64[ns]
user              24163 non-null object
dtypes: bool(2), datetime64[ns](1), int64(2), object(7)
memory usage: 1.9+ MB


In [179]:
df.text=df.text.apply(lambda x: " ".join(x.split()) if (x==x)&(type(x)==unicode) else None)
df.to_csv("baseline_revisions.csv",sep="\t")

## Count words in revision text

In [196]:
df["text_len"]=df.text.apply(lambda x: len(TextBlob(x).words)  if (x==x)&(type(x)==unicode) else None)

In [197]:
df.head(2)

,_content_model,_parent_id,_sha1,anon,article,comment,minor,revid,rollbacktoken,text,timestamp,user,text_len
0,wikitext,807253069,a0d1ed8b580892cc7dabf3d770bc69d60bed5fc7,False,G%C3%A1bor_A._Somorjai,Grammar correction,True,811195141,None,{{Infobox person |name = Gábor A. Somorjai |im...,2017-11-20 02:30:05,HandsomeMrToad,1593.0
1,wikitext,797794813,d0c871038a54950f633bb93b5f73ed1a820a7a58,False,G%C3%A1bor_A._Somorjai,Rescuing 3 sources and tagging 0 as dead. #IAB...,False,807253069,None,{{Infobox person |name = Gábor A. Somorjai |im...,2017-10-26 21:09:27,InternetArchiveBot,1594.0


In [200]:
df.to_csv("baseline_revisions.csv",sep="\t")

## Awarded scientists

In [187]:
list_rev2=[]
erorr_list2=[]

j=1
for sci_name in sci_aw_dict.keys():
    name=sci_name.split("/")[-1]
    print j,name
    try:
        page = pywikibot.Page(site, name)
        revs = page.revisions(content=True)
        for i in revs:
            temp_dict=i.__dict__
            temp_dict["article"]=name
            list_rev2.append(temp_dict)
    except:
        print "Error when access:", name
        erorr_list2.append(name)
        
    j+=1
    
df2=pd.DataFrame(list_rev2)
df2.info()

df2.text=df2.text.apply(lambda x: " ".join(x.split()) if (x==x)&(type(x)==unicode) else None)
df2.to_csv("seed_revisions.csv",sep="\t")

1 Michael_Grunstein
2 Bernd_Giese
3 John_Tate
4 Nobuhiro_Kiyotaki
5 Shinya_Yamanaka
6 Richard_Hynes
7 Mikhail_Leonidovich_Gromov
8 Charles_L._Bennett
9 David_Julius
10 C%C3%A9dric_Villani
11 Nancy_A._Moran
12 Peter_Howitt_(economist)
13 Steven_Van_Slyke
14 Hideo_Hosono
15 Barbara_Liskov
16 Ching_W._Tang
17 Charles_Sawyers
18 Thomas_Ebbesen
19 Tu_Youyou
20 Irwin_M._Jacobs
21 Martin_Hairer
22 Jeffrey_M._Friedman
23 Leslie_Lamport
24 Ferenc_Krausz
25 William_Nordhaus
26 Paul_Corkum
27 John_G._Thompson
28 Jack_W._Szostak
29 Bruce_Beutler
30 John_Milnor
31 Matthew_Rabin
32 Charles_F._Manski
33 M._Hashem_Pesaran
34 Robert_Lefkowitz
35 Endre_Szemer%C3%A9di
36 Patrick_O._Brown
37 Harold_Demsetz
38 Sam_Peltzman
39 Aziz_Sancar
40 James_Rothman
41 Anne_Osborn_Krueger
42 Alan_Krueger
43 Tony_Atkinson
44 John_B._Goodenough
45 Seiji_Ogawa
46 Peter_C._B._Phillips
47 David_R._Smith
48 Charles_L._Kane
49 Ernest_McCulloch
50 Eric_H._Davidson
51 Jacques_Tits
52 Shizuo_Akira
53 John_A._List
54 Paul_Alivis

In [188]:
print erorr_list2

[]


In [198]:
df2["text_len"]=df2.text.apply(lambda x: len(TextBlob(x).words)  if (x==x)&(type(x)==unicode) else None)

In [201]:
df2.to_csv("seed_revisions.csv",sep="\t")

# convert wiki to plain text

In [156]:
#pip install mediawiki-parser
#https://github.com/peter17/mediawiki-parser 



from mediawiki_parser.preprocessor import make_parser
from mediawiki_parser.text import make_parser

templates = {}
preprocessor = make_parser(templates)
parser = make_parser()



df["text_clean"]=df.text.apply(lambda x: parser.parse(preprocessor.parse(x).leaves()).value.strip())

KeyboardInterrupt: 